In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thyroid_Cancer"
cohort = "GSE76039"

# Input paths
in_trait_dir = "../../input/GEO/Thyroid_Cancer"
in_cohort_dir = "../../input/GEO/Thyroid_Cancer/GSE76039"

# Output paths
out_data_file = "../../output/preprocess/Thyroid_Cancer/GSE76039.csv"
out_gene_data_file = "../../output/preprocess/Thyroid_Cancer/gene_data/GSE76039.csv"
out_clinical_data_file = "../../output/preprocess/Thyroid_Cancer/clinical_data/GSE76039.csv"
json_path = "../../output/preprocess/Thyroid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genomic and Transcriptomic Hallmarks of Poorly-Differentiated and Anaplastic Thyroid Cancers"
!Series_summary	"BACKGROUND. Poorly-differentiated (PDTC) and anaplastic (ATC) thyroid cancers are rare and frequently lethal tumors, which so far have not been subjected to comprehensive genetic characterization.  METHODS. We performed next generation sequencing of 341 cancer genes in 117 PDTCs and ATCs, and a transcriptomic analysis of a representative subset of 37 tumors. Results were analyzed in the context of The Cancer Genome Atlas (TCGA) study of papillary thyroid cancers (PTC).  RESULTS. ATCs have a greater mutation burden than PDTCs, and higher mutation frequency of TP53, TERT promoter, PI3K/AKT/mTOR pathway effectors, SWI/SNF subunits and histone methyltransferases. BRAF and RAS are the predominant drivers, and dictate remarkably distinct tropism for nodal vs. distant metastases in PDTC. RAS and BRAF sharply distinguish between PDTCs defined by 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this is an Affymetrix U133 plus 2.0 array study
# which is a gene expression microarray, so gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Trait Data
# From the sample characteristics, we can identify the tumor type row (2) as containing the trait information
trait_row = 2

def convert_trait(value_str):
    # Extract the value after the colon and strip whitespace
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip()
        
        # Binary categorization: primary tumor (0) vs any other tumor type (1)
        if value.lower() == 'primary' or value.lower() == 'primary (residual)':
            return 0  # Primary tumor
        else:
            return 1  # Other tumor types (metastasis, recurrent, etc.)
    return None

# 2.2 Age Data
# Age data is not available in the sample characteristics
age_row = None

def convert_age(value_str):
    # This function won't be used since age_row is None
    return None

# 2.3 Gender Data
# Gender information is available in row 0
gender_row = 0

def convert_gender(value_str):
    # Extract the value after the colon and strip whitespace
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip().lower()
        
        # Convert gender to binary (female=0, male=1)
        if value == 'female':
            return 0
        elif value == 'male':
            return 1
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with feature extraction
if trait_row is not None:
    # We'll skip trying to load the clinical_data.csv file as it doesn't exist
    # Instead, we'll wait for a subsequent step where the clinical data will be available
    print("Clinical data extraction will be performed in a subsequent step when the data is available.")
    
    # Note: We're not running the geo_select_clinical_features function here
    # as we don't have the clinical_data DataFrame yet
    
    # Create directory for future clinical data file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)


Clinical data extraction will be performed in a subsequent step when the data is available.


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Thyroid_Cancer/GSE76039/GSE76039_family.soft.gz
Matrix file: ../../input/GEO/Thyroid_Cancer/GSE76039/GSE76039_series_matrix.txt.gz
Found the matrix table marker in the file.


Gene data shape: (54675, 37)
First 20 gene/probe identifiers:
['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at', '1552263_at', '1552264_a_at', '1552266_at']


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (like '1007_s_at', '1053_at', etc.) appear to be Affymetrix probe IDs,
# not standard human gene symbols. Affymetrix probe IDs need to be mapped to official gene symbols
# for proper biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3].to_string())

# Check for columns that might contain gene information
potential_gene_columns = [col for col in gene_annotation.columns if 
                          any(term in col.upper() for term in ["GENE", "SYMBOL", "NAME", "ID"])]
print(f"\nPotential gene-related columns: {potential_gene_columns}")

# Correctly identify the columns for probe ID and gene symbols
gene_id_col = 'ID'  # This is the probe identifier column
gene_symbol_col = 'Gene Symbol'  # Corrected to match the actual column name in the data

# Verify columns exist before mapping
if gene_id_col in gene_annotation.columns and gene_symbol_col in gene_annotation.columns:
    # Create the mapping using the library function
    mapping_data = get_gene_mapping(gene_annotation, gene_id_col, gene_symbol_col)
    
    # Print sample of the mapping to confirm
    print("\nSample of probe ID to gene symbol mappings:")
    print(mapping_data.head(10))
    
    # Check the size of the mapping data
    print(f"\nTotal number of probe-to-gene mappings: {len(mapping_data)}")
else:
    missing_cols = []
    if gene_id_col not in gene_annotation.columns:
        missing_cols.append(gene_id_col)
    if gene_symbol_col not in gene_annotation.columns:
        missing_cols.append(gene_symbol_col)
    print(f"\nError: The following columns are missing from the annotation data: {missing_cols}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Get gene expression data from the matrix file
gene_expression_data = get_genetic_data(matrix_file)

# 3. Get gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# 4. Identify columns for probe ID and gene symbol in the annotation data
gene_id_col = 'ID'        # Column that matches identifiers in the gene expression data
gene_symbol_col = 'Gene Symbol'  # Column containing the gene symbols

# 5. Create gene mapping dataframe
mapping_data = get_gene_mapping(gene_annotation, gene_id_col, gene_symbol_col)

# 6. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_expression_data, mapping_data)

# Print information about the conversion
print(f"Original probe data shape: {gene_expression_data.shape}")
print(f"Mapping data shape: {mapping_data.shape}")
print(f"Mapped gene data shape: {gene_data.shape}")
print(f"Number of unique gene symbols after mapping: {len(gene_data.index.unique())}")

# Preview the first few rows of the gene data
print("\nPreview of gene expression data:")
print(gene_data.head())


Original probe data shape: (54675, 37)
Mapping data shape: (45782, 2)
Mapped gene data shape: (21278, 37)
Number of unique gene symbols after mapping: 21278

Preview of gene expression data:
          GSM2024824  GSM2024825  GSM2024826  GSM2024827  GSM2024828  \
Gene                                                                   
A1BG        5.610503    5.636573    4.251042    5.007339    4.243692   
A1BG-AS1    4.479533    4.360212    3.876465    4.364013    2.777218   
A1CF        4.522775    4.522775    4.522775    4.522775    4.522775   
A2M        12.425145   11.317791   14.370494   12.110579   10.911067   
A2M-AS1     3.488812    3.172215    3.292311    4.061210    3.314219   

          GSM2024829  GSM2024830  GSM2024831  GSM2024832  GSM2024833  ...  \
Gene                                                                  ...   
A1BG        3.668276    5.043683    6.892920    6.381826    6.931566  ...   
A1BG-AS1    3.374334    4.124470    4.215001    4.396431    3.876242  ...

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data with correct row indices
try:
    # Load the original clinical data (from Step 1)
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Extract clinical features with correct indices from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=2,      # Tumor type row from Step 2
        convert_trait=convert_trait,
        gender_row=0,     # Gender row from Step 2
        convert_gender=convert_gender,
        age_row=None,     # Age data is not available as per Step 2
        convert_age=None
    )
    
    print(f"Extracted clinical data shape: {clinical_features.shape}")
    print("Preview of clinical data (first 5 samples):")
    print(clinical_features.iloc[:, :5])
    
    # Save the properly extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from thyroid cancer samples with tumor type information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note="Missing essential data components for linking (trait data or gene expression data)."
    )

Gene data shape before normalization: (21278, 37)
Gene data shape after normalization: (19845, 37)


Normalized gene data saved to ../../output/preprocess/Thyroid_Cancer/gene_data/GSE76039.csv
Extracted clinical data shape: (2, 37)
Preview of clinical data (first 5 samples):
                GSM2024824  GSM2024825  GSM2024826  GSM2024827  GSM2024828
Thyroid_Cancer         0.0         0.0         0.0         0.0         0.0
Gender                 0.0         0.0         1.0         0.0         1.0
Clinical data saved to ../../output/preprocess/Thyroid_Cancer/clinical_data/GSE76039.csv
Gene data columns (first 5): ['GSM2024824', 'GSM2024825', 'GSM2024826', 'GSM2024827', 'GSM2024828']
Clinical data columns (first 5): ['GSM2024824', 'GSM2024825', 'GSM2024826', 'GSM2024827', 'GSM2024828']
Found 37 common samples between gene and clinical data
Initial linked data shape: (37, 19847)
Preview of linked data (first 5 rows, first 5 columns):
            Thyroid_Cancer  Gender      A1BG  A1BG-AS1      A1CF
GSM2024824             0.0     0.0  5.610503  4.479533  4.522775
GSM2024825             0.0 

Linked data shape after handling missing values: (37, 19847)
For the feature 'Thyroid_Cancer', the least common label is '1.0' with 7 occurrences. This represents 18.92% of the dataset.
The distribution of the feature 'Thyroid_Cancer' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 10 occurrences. This represents 27.03% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Thyroid_Cancer/GSE76039.csv
